In [1]:
import pandas as pd
import numpy as np
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
## for processing
import re
import nltk
## for bag-of-words
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing,feature_selection
## for explainer
from lime import lime_text
## for word embedding
import gensim
import gensim.downloader as gensim_api
## for deep learning
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K
## for bert language model
import transformers

In [2]:
from sklearn import feature_selection
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder

In [3]:
review=pd.read_csv('Review_project_sentiment.csv')
review.head(3)

,Unnamed: 0,review_id,user_id,business_id,stars,useful,funny,cool,text,date,useful_level,language,text_clean,read_ease,polarity,subjectivity,Adj_ratio
0,1073206,JdReKgETiiJEDmshrO4TLw,pyarmAnR-i-qookQamqRTA,V2GOReqPvr8qpCC7sWfoTw,1.0,17,1,0,Just to let this car company that people DO re...,2014-03-06 12:38:52,moderate useful,en,let car company people read yelp check review ...,85.49,-0.500000,1.000000,0.120000
1,6229216,zL4se_Ixdcl8kvTOHCS3rg,s16-BUo-orUsELvMu5ocKg,VH0Ib9S3E-dxbQdQC4rffg,1.0,15,6,4,Mistral was the worst dining experience I have...,2010-07-22 18:08:01,moderate useful,en,mistral worst dining experience ever life bad ...,79.19,0.029864,0.502499,0.225888
2,7607157,wlLQ42QMHiQMJISCTUVX5A,aQA2eIdEC3_ZDHwUregu8A,wygOtij7aEb7dTdTFFnOcw,5.0,0,0,0,Took a VIP tour with DeMon of the Atlanta Merc...,2020-02-04 16:05:20,not useful,en,took vip tour demon atlanta mercedes benz stad...,-32.40,0.422500,0.696111,0.218750


In [25]:
#encoder
review1=review
review1.loc[review1['useful_level']=="useful",'useful_level']=2
review1.loc[review1['useful_level']=="moderate useful",'useful_level']=1
review1.loc[review1['useful_level']=="not useful",'useful_level']=0

#encoder1=LabelEncoder()
#review1['useful_level'] = encoder1.fit_transform(review1['useful_level'].values)
review1.head()

,Unnamed: 0,review_id,user_id,business_id,stars,useful,funny,cool,text,date,useful_level,language,text_clean,read_ease,polarity,subjectivity,Adj_ratio
0,1073206,JdReKgETiiJEDmshrO4TLw,pyarmAnR-i-qookQamqRTA,V2GOReqPvr8qpCC7sWfoTw,1.0,17,1,0,Just to let this car company that people DO re...,2014-03-06 12:38:52,1,en,let car company people read yelp check review ...,85.49,-0.500000,1.000000,0.120000
1,6229216,zL4se_Ixdcl8kvTOHCS3rg,s16-BUo-orUsELvMu5ocKg,VH0Ib9S3E-dxbQdQC4rffg,1.0,15,6,4,Mistral was the worst dining experience I have...,2010-07-22 18:08:01,1,en,mistral worst dining experience ever life bad ...,79.19,0.029864,0.502499,0.225888
2,7607157,wlLQ42QMHiQMJISCTUVX5A,aQA2eIdEC3_ZDHwUregu8A,wygOtij7aEb7dTdTFFnOcw,5.0,0,0,0,Took a VIP tour with DeMon of the Atlanta Merc...,2020-02-04 16:05:20,0,en,took vip tour demon atlanta mercedes benz stad...,-32.40,0.422500,0.696111,0.218750
3,141509,OMw52-Eo-BYFHdUldLMc5Q,mmr-P0i__uQJHGXXeVjXxg,0Ng-wDNyA-96uaJZP8EKZg,4.0,40,0,0,When I first came here for the first time last...,2015-03-09 22:42:06,2,en,first came first time last year best friend ap...,83.96,0.168452,0.419841,0.168831
4,7962637,RWq78T9FIyauVHaHj7yl9g,CyCI71vLDa923zOMeDDA5w,WUfZXjTtBUbezJE7LnWABw,3.0,0,0,0,"I only got the take-out, so I cannot speak of ...",2013-03-22 20:47:40,0,en,got takeout cannot speak service food good wou...,51.86,0.320000,0.780000,0.200000


In [80]:
review1.to_csv('review_sentiment_encoded.csv')

从这里开始

In [3]:
review1=pd.read_csv(r'C:\Users\11638\Favorites\Code\Project\fan\Data\review_sentiment_encoded.csv')

In [4]:
## Tf-Idf (advanced variant of BoW)
vectorizer = feature_extraction.text.TfidfVectorizer(max_features=5000, ngram_range=(1,1))

In [5]:
## split dataset
dtf_train, dtf_test = model_selection.train_test_split(review1, test_size=0.3)
## get target
y_train = dtf_train["useful_level"].values
y_test = np.array(dtf_test["useful_level"].values)

In [6]:
corpus1 = dtf_train["text_clean"]
vectorizer.fit(corpus1)
X_train = vectorizer.transform(corpus1)
dic_vocabulary = vectorizer.vocabulary_

In [7]:
X_train.shape

(38630, 5000)

In [8]:
y = dtf_train["useful_level"]
X_names = vectorizer.get_feature_names()
p_value_limit = 0.95
dtf_features = pd.DataFrame()
for cat in np.unique(y):
    chi2, p = feature_selection.chi2(X_train, y==cat)
    dtf_features = dtf_features.append(pd.DataFrame(
                   {"feature":X_names, "score":1-p, "y":cat}))
    dtf_features = dtf_features.sort_values(["y","score"], 
                    ascending=[True,False])
    dtf_features = dtf_features[dtf_features["score"]>p_value_limit]
X_names = dtf_features["feature"].unique().tolist()

In [9]:
for cat in np.unique(y):
    print("# {}:".format(cat))
    print("  . selected features:",
         len(dtf_features[dtf_features["y"]==cat]))
    print("  . top features:", ",".join(
dtf_features[dtf_features["y"]==cat]["feature"].values[:10]))
    print(" ")

# 0:
  . selected features: 1545
  . top features: amazing,company,food,great,atmosphere,call,told,doctor,pizza,review
 
# 1:
  . selected features: 557
  . top features: great,car,food,apartment,call,said,office,told,doctor,amazing
 
# 2:
  . selected features: 431
  . top features: ride,park,food,ron,great,disney,tattoo,store,grandson,lash
 


In [58]:
# refit vecorizor
vectorizer = feature_extraction.text.TfidfVectorizer(vocabulary=X_names)
vectorizer.fit(corpus1)
X_train = vectorizer.transform(corpus1)
dic_vocabulary = vectorizer.vocabulary_

In [59]:
corpus2 = dtf_test["text_clean"]
X_test= vectorizer.transform(corpus2)

# 停

In [12]:
classifier = naive_bayes.MultinomialNB()

In [13]:
## pipeline
model = pipeline.Pipeline([("vectorizer", vectorizer),  
                           ("classifier", classifier)])
## train classifier
model["classifier"].fit(X_train, y_train.astype(int))
## test
X_test = dtf_test["text_clean"].values
predicted = model.predict(X_test)
#predicted = np.argmax(predicted,axis=1)
predicted_prob = model.predict_proba(X_test)

In [14]:
from sklearn.metrics import f1_score, precision_score, recall_score
f1 = f1_score( y_test, predicted, average='macro' )
p = precision_score(y_test, predicted, average='macro')
r = recall_score(y_test, predicted, average='macro')

print(f1, p, r)
# output: 0.555555555556 0.5 0.666666666667


0.4933844182710021 0.5495284065115231 0.5079293973412745


In [15]:
from sklearn.metrics import confusion_matrix

In [16]:
accuracy = metrics.accuracy_score(y_test, predicted)
auc = metrics.roc_auc_score(y_test, predicted_prob, 
                            multi_class="ovr")
print("Accuracy:",  round(accuracy,2))
print("Auc:", round(auc,2))
print("Detail:")
print(metrics.classification_report(y_test, predicted))

Accuracy: 0.57
Auc: 0.73
Detail:
              precision    recall  f1-score   support

           0       0.62      0.82      0.71      7327
           1       0.49      0.54      0.51      5222
           2       0.53      0.17      0.26      4008

    accuracy                           0.57     16557
   macro avg       0.55      0.51      0.49     16557
weighted avg       0.56      0.57      0.54     16557



# 重新开始

## 合并

In [103]:
AllotherTrain = dtf_train[["useful_level", "read_ease", "polarity", "subjectivity", "Adj_ratio"]]

In [104]:
EmbeddingTrain = pd.DataFrame(X_train.toarray(), columns = vectorizer.get_feature_names())

In [105]:
for i in ['useful_level','read_ease','polarity','subjectivity','Adj_ratio']:
    EmbeddingTrain[i] = AllotherTrain[i].values

In [106]:
EmbeddingTrain

,amazing,company,food,great,atmosphere,call,told,doctor,pizza,review,...,nicely,popped,overpriced,tourist,caribbean,useful_level,read_ease,polarity,subjectivity,Adj_ratio
0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0,75.71,0.382955,0.484848,0.133333
1,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0,82.14,-0.266667,0.400000,0.111111
2,0.030002,0.034298,0.000000,0.000000,0.0,0.0,0.027971,0.0,0.0,0.027081,...,0.0,0.0,0.0,0.0,0.0,2,-42.14,0.099065,0.468141,0.179487
3,0.000000,0.103333,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,1,42.92,0.163417,0.503500,0.101695
4,0.000000,0.121432,0.000000,0.000000,0.0,0.0,0.066021,0.0,0.0,0.319599,...,0.0,0.0,0.0,0.0,0.0,2,-25.29,0.092721,0.489664,0.147887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38625,0.000000,0.000000,0.000000,0.211778,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0,52.53,0.355000,0.565000,0.352941
38626,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,1,72.02,0.050000,0.775000,0.133333
38627,0.000000,0.000000,0.073666,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,2,67.18,0.157917,0.442302,0.277778
38628,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0,85.18,0.194643,0.552381,0.225000


In [107]:
NewXTrain = EmbeddingTrain.loc[:, EmbeddingTrain.columns != 'useful_level']

In [108]:
NewYTrain = EmbeddingTrain.loc[:, EmbeddingTrain.columns == 'useful_level']

In [109]:
#NewYTrain = EmbeddingTrain.loc[:, EmbeddingTrain.columns == 'useful_level'].values
#NewYTrain = np.ravel(NewYTrain)

In [110]:
AllotherTest = dtf_test[["useful_level", "read_ease", "polarity", "subjectivity", "Adj_ratio"]]

In [111]:
EmbeddingTest = pd.DataFrame(X_test.toarray(), columns = vectorizer.get_feature_names())

In [112]:
for i in ['useful_level','read_ease','polarity','subjectivity','Adj_ratio']:
    EmbeddingTest[i] = AllotherTest[i].values

In [113]:
NewXTest = EmbeddingTest.loc[:, EmbeddingTest.columns != 'useful_level']
NewYTest = EmbeddingTest.loc[:, EmbeddingTest.columns == 'useful_level'].values

In [114]:
#NewYTest = EmbeddingTest.loc[:, EmbeddingTest.columns == 'useful_level'].values
#NewYTest = np.ravel(NewYTest)

In [115]:
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn import svm
from sklearn.metrics import accuracy_score


In [118]:
logreg = LogisticRegression(max_iter = 5000)
logreg.fit(NewXTrain, NewYTrain)
print('Accuracy of Logistic regression classifier on training set: {:.2f}'
     .format(logreg.score(NewXTrain, NewYTrain)))
print('Accuracy of Logistic regression classifier on test set: {:.2f}'
     .format(logreg.score(NewXTest, NewYTest)))

C:\Users\11638\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Accuracy of Logistic regression classifier on training set: 0.67
Accuracy of Logistic regression classifier on test set: 0.62


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(NewXTrain, NewYTrain)
print('Accuracy of K-NN classifier on training set: {:.2f}'
     .format(knn.score(NewXTrain, NewYTrain)))
print('Accuracy of K-NN classifier on test set: {:.2f}'
     .format(knn.score(NewXTest, NewYTest)))

In [119]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lda.fit(NewXTrain, NewYTrain)
print('Accuracy of LDA classifier on training set: {:.2f}'
     .format(lda.score(NewXTrain, NewYTrain)))
print('Accuracy of LDA classifier on test set: {:.2f}'
     .format(lda.score(NewXTest, NewYTest)))

C:\Users\11638\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Accuracy of LDA classifier on training set: 0.67
Accuracy of LDA classifier on test set: 0.62


In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(NewXTrain, NewYTrain)
print('Accuracy of GNB classifier on training set: {:.2f}'
     .format(gnb.score(NewXTrain, NewYTrain)))
print('Accuracy of GNB classifier on test set: {:.2f}'
     .format(gnb.score(NewXTest, NewYTest)))

In [ ]:
from sklearn.svm import SVC
svm = SVC()
svm.fit(NewXTrain, NewYTrain)
print('Accuracy of SVM classifier on training set: {:.2f}'
     .format(svm.score(NewXTrain, NewYTrain)))
print('Accuracy of SVM classifier on test set: {:.2f}'
     .format(svm.score(NewXTest, NewYTest)))